## Populating data

### songs nested into playlists

In [1]:
import pandas as pd
from pymongo import MongoClient

songs_df = pd.read_csv('https://raw.githubusercontent.com/Byzon777/ForgeEngineers/refs/heads/main/Tables/songs.csv')
playlists_df = pd.read_csv('https://raw.githubusercontent.com/Byzon777/ForgeEngineers/refs/heads/main/Tables/playlists.csv')

# split song id's by commas
def parse_song_ids(s):
    return [sid.strip() for sid in str(s).split(',') if sid.strip()]

# dic of song id and song info
songs_by_id = songs_df.set_index('song_id').to_dict(orient='index')

In [3]:
# loop through each row in playlists
playlist_docs = []
for _, row in playlists_df.iterrows():
    song_ids = parse_song_ids(row['Songs'])
    nested_songs = [songs_by_id[sid] for sid in song_ids if sid in songs_by_id] # getting song info for each song id
    
    # making document structure
    playlist_doc = {
        'playlist_name': row['Playlist_name'],
        'user_id':       row['User_id'],
        'genre':         row['Genre'],
        'listens':       row['Listens'],
        'songs':         nested_songs
    }
    playlist_docs.append(playlist_doc)

In [4]:
# populate in mongo
client = MongoClient('mongodb://localhost:27017/')
db = client['music_database'] # music database
playlists_col = db['playlists'] # playlist collection
result = playlists_col.insert_many(playlist_docs)

### artist, parent company, songs nested into albums

In [5]:
albums_df  = pd.read_csv('https://raw.githubusercontent.com/Byzon777/ForgeEngineers/refs/heads/main/Tables/albums.csv')
artists_df = pd.read_csv('https://raw.githubusercontent.com/Byzon777/ForgeEngineers/refs/heads/main/Tables/artists.csv')

# merge albums and artists on artist id
merged_df = albums_df.merge(
    artists_df,
    on='artist_id',
    suffixes=('', '_artist')
)

In [6]:
parent_df = pd.read_csv('https://raw.githubusercontent.com/Byzon777/ForgeEngineers/refs/heads/main/Tables/parent_company.csv')

# get rid of spaces
parent_df.columns = parent_df.columns.str.strip()
parent_df = parent_df.rename(columns={'Album Id': 'album_id'}) # rename column

# add parent to merged df
merged_df = merged_df.merge(
    parent_df,
    on='album_id',
    suffixes=('', '_parent'),
    how='left'
)

In [7]:
merged_df.head()

,album_id,name,artist_name,artist_id,year,genre,track_count,revenue (mln USD),name_artist,city,country,followers,gender,Artist,Year of Release,Parent Company
0,AL001,Midnights,Taylor Swift,A001,2022,Pop,13,194.0,Taylor Swift,Reading,USA,"92,000,000",Female,Taylor Swift,2022,Universal Music Group
1,AL002,Un Verano Sin Ti,Bad Bunny,A002,2022,Reggaeton,23,476.0,Bad Bunny,San Juan,Puerto Rico,"72,000,000",Male,Bad Bunny,2022,Rimas Entertainment
2,AL003,BE,BTS,A003,2020,K-Pop,8,369.0,BTS,Seoul,South Korea,"75,000,000",Male,BTS,2020,HYBE Corporation
3,AL004,Happier Than Ever,Billie Eilish,A004,2021,Pop,16,303.0,Billie Eilish,Los Angeles,USA,"68,000,000",Female,Billie Eilish,2021,Universal Music Group
4,AL005,Future Nostalgia,Dua Lipa,A005,2020,Pop,11,86.0,Dua Lipa,London,UK,"58,000,000",Female,Dua Lipa,2020,Warner Music Group


In [8]:
songs_df = pd.read_csv('https://raw.githubusercontent.com/Byzon777/ForgeEngineers/refs/heads/main/Tables/songs.csv')
songs_by_album = (
    songs_df
    .groupby('album name')
    .apply(lambda df: df.to_dict(orient='records'))
    .to_dict()
)

/var/folders/pw/r039mjv14tx5v58l0m9yjvkc0000gn/T/ipykernel_39691/3045886409.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda df: df.to_dict(orient='records'))


In [9]:
documents = []
for _, row in merged_df.iterrows():
    # album document
    album_doc = { col: row[col] for col in albums_df.columns }

    # nested artist dict
    artist_doc = {
        col: row[f"{col}_artist"] if f"{col}_artist" in row else row[col]
        for col in artists_df.columns
    }
    album_doc['artist'] = artist_doc

    # nested parent company dict
    parent_doc = {
        col: row[f"{col}_parent"] if f"{col}_parent" in row else row[col]
        for col in parent_df.columns
    }
    album_doc['parent_company'] = parent_doc

    # nested songs dict
    album_title = row['name']
    album_doc['songs'] = songs_by_album.get(album_title, [])

    documents.append(album_doc)


In [10]:
# populate in mongo
client = MongoClient('mongodb://localhost:27017/')
db = client['music_database']
albums_col = db['albums']
result = albums_col.insert_many(documents)

## Querying

In [26]:
# Which album had the most revenue
top_rev_album = albums_col.find_one(sort=[("revenue (mln USD)", -1)])
print(top_rev_album['name'])
print(top_rev_album['artist_name'])
print(str(top_rev_album['revenue (mln USD)'])+ " million USD")

The Kids Are Coming
Tones and I
493.57 million USD
